In [1]:
import os
import sys
import time

import GCode
import GRBL
import matplotlib.pyplot as plt
import numpy as np

# Draw a Brick Pattern

Attempt at programmatically making a brick pattern.

All units in mm. ```1``` = ```1 mm```.

"Napkin" scratches.

![](http://luckofthedraw.fun/.imgs/brick_0003.jpeg)

Drawn by hand. ~18mm brick height.

![](http://luckofthedraw.fun/.imgs/brick_0001.jpeg)

> Standard bricks. The standard co-ordinating size for brickwork is 225 mm x 112.5 mm x 75 mm (length x depth x height). This includes 10 mm mortar joints, and so the standard size for a brick itself is 215 mm x 102.5 mm x 65 mm (length x depth x height).

In [2]:
# Standard brick dimensions.
BrickHeight = 65  # [mm]
BrickLength = 225  # [mm]
BrickDepth = 12.5  # [mm]
BrickRatio = 215 / 65  # [dimensionless]


In [3]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [4]:
# Drawing configuration.
# How many rows of bricks to draw on the block.
N_BrickRows = 12  # [dimensionless]

# Dimensions of a 'brick' projected onto the block of wood.
H_Block_Brick = BlockHeight / N_BrickRows  # [mm]
L_Block_Brick = H_Block_Brick * BrickRatio  # [mm]


# Code:

In [5]:
flip = np.array([[1, 1], [1, 0]])
transform_tuple = (
    np.eye(2),  # Identity matrix, do nothing.
    np.eye(2),  # Do nothing, for debugging.
    flip,  # Flip the matrix, reduces travel time.
)
vertical_brick_lines_tuple = (
    np.arange(L_Block_Brick, BlockLength, L_Block_Brick),  # Odd rows.
    np.arange(L_Block_Brick / 2, BlockLength, L_Block_Brick),  # Even rows.
)
horizontal_brick_lines = np.linspace(
    0, BlockHeight, N_BrickRows, endpoint=False
)


Lines parallel to the X-axis.
Separates rows of bricks.

In [6]:
horizontal_brick_lines


array([ 0.        ,  7.41666667, 14.83333333, 22.25      , 29.66666667,
       37.08333333, 44.5       , 51.91666667, 59.33333333, 66.75      ,
       74.16666667, 81.58333333])

Lines parallel to the Y-axis.

In [7]:
vertical_brick_lines_tuple


(array([ 24.53205128,  49.06410256,  73.59615385,  98.12820513,
        122.66025641, 147.19230769, 171.72435897]),
 array([ 12.26602564,  36.79807692,  61.33012821,  85.86217949,
        110.39423077, 134.92628205, 159.45833333]))

In [8]:
BlockBrick = GCode.Program()
BlockBrick.lines = list()

for idx in range(1, len(horizontal_brick_lines)):
    # Top horizontal line that defines each 'brick'
    horizontal_brick_line = horizontal_brick_lines[idx]
    row_line_points = np.array(
        [[0, horizontal_brick_line], [BlockLength, horizontal_brick_line]]
    )
    # Transform to perform on the row points.
    transform = transform_tuple[np.mod(idx, 2)]

    row_line_points = np.matmul(transform, row_line_points)
    line_ = GCode.Line(points=row_line_points)

    BlockBrick.lines.append(line_)

    # Vertical brick line.
    vertical_brick_lines = vertical_brick_lines_tuple[
        np.mod(idx, len(vertical_brick_lines_tuple))
    ]

    start_point_y = horizontal_brick_lines[idx - 1]
    end_point_y = horizontal_brick_lines[idx]

    for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
        transform = transform_tuple[np.mod(idx2, 2)]
        column_line_points = (
            np.array(
                [
                    [vertical_brick_line, start_point_y],
                    [vertical_brick_line, end_point_y],
                ]
            ),
        )
        column_line_points = np.matmul(transform, column_line_points)
        line_ = GCode.Line(column_line_points)
        BlockBrick.lines.append(line_)

# Not drawing the last row of vertical lines.
idx += 1
vertical_brick_lines = vertical_brick_lines_tuple[
    np.mod(idx, len(vertical_brick_lines_tuple))
]

start_point_y = horizontal_brick_lines[idx - 1]
end_point_y = BlockHeight

for idx2, vertical_brick_line in enumerate(vertical_brick_lines):
    transform = transform_tuple[np.mod(idx2, 2)]
    column_line_points = (
        np.array(
            [
                [vertical_brick_line, start_point_y],
                [vertical_brick_line, end_point_y],
            ]
        ),
    )
    column_line_points = np.matmul(transform, column_line_points)
    line_ = GCode.Line(column_line_points)
    BlockBrick.lines.append(line_)


In [9]:
BlockBrick.lines


[Line<len=178.0mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=178.0mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=178.0mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667mm, feed=300, power=150>,
 Line<len=7.41667

In [10]:
BlockBrick.time


3313.6085462389365

In [11]:
BlockBrick.dist


8175.817932477876

In [12]:
BlockBrick.generate_gcode()


In [15]:
BlockBrick.save("12row_brick_2x1.gcode")

In [18]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
cnc.reset()


(4, ['', "Grbl 1.1f ['$' for help]", "[MSG:'$H'|'$X' to unlock]"])

In [19]:
cnc.status


'<Alarm|MPos:0.000,0.000,0.000|Bf:15,127|FS:0,0|WCO:0.000,0.000,0.000>'

In [20]:
cnc.home()


14

In [21]:
cnc.status


'<Idle|MPos:-259.000,-174.000,-1.000|Bf:15,127|FS:0,0|Ov:100,100,100>'

In [22]:
BlockBrick.machine = cnc


In [34]:
BlockBrick.run()
